In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [3]:

#1.位置编码
class PositionalEncoding(nn.Module):  # 继承 PyTorch 的 nn.Module
    def __init__(self, d_model, max_len=5000):  # d_model 是词嵌入维度，max_len 是序列的最大长度
        super().__init__()  # 调用父类 nn.Module 的构造函数

        # 创建一个形状为 (max_len, d_model) 的零矩阵，用于存储位置编码
        pe = torch.zeros(max_len, d_model)

        # 生成一个形状为 (max_len, 1) 的张量，表示序列中的每个位置
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # 计算分母项，使用 e^( - (2i / d_model) * log(10000) )
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # 计算 sin 值并赋值给偶数索引（0, 2, 4, ...）的维度
        pe[:, 0::2] = torch.sin(position * div_term)

        # 计算 cos 值并赋值给奇数索引（1, 3, 5, ...）的维度
        pe[:, 1::2] = torch.cos(position * div_term)

        # 添加 batch 维度，使形状变为 (1, max_len, d_model)
        pe = pe.unsqueeze(0)

        # 将 pe 注册为 buffer，使其在训练过程中不会被当作模型参数更新
        self.register_buffer('pe', pe)

    def forward(self, x):
        # 将位置编码加到输入 x 上，x 的形状为 (batch_size, seq_len, d_model)
        return x + self.pe[:, :x.size(1)]  # 只取与输入序列长度匹配的部分


In [4]:
positional_encoding=PositionalEncoding(512)
x=torch.randn(2,512,512)
print(x)
x=positional_encoding(x)
print(x)

tensor([[[ 0.7195,  0.1422, -0.3647,  ...,  0.9219,  0.6199, -0.0868],
         [-1.5566, -0.1231,  0.7637,  ..., -0.1128,  0.2161,  0.7489],
         [-0.9458, -0.0855, -0.7524,  ..., -1.9013,  1.7498,  0.1070],
         ...,
         [-0.0923,  0.4496,  0.2249,  ...,  0.4902, -0.6342, -0.7123],
         [ 0.1207,  0.9847,  1.2898,  ...,  1.3538, -0.2600,  0.6467],
         [-1.1969,  0.6190,  0.1341,  ...,  0.6129, -0.2538,  0.2888]],

        [[-0.0290,  1.3108,  0.1407,  ..., -0.2343, -1.0975,  0.2140],
         [-0.8824,  0.5006, -1.7399,  ..., -1.3901, -0.7364, -0.4741],
         [ 2.1290,  1.3590,  1.9984,  ..., -0.0939, -1.7093,  0.2820],
         ...,
         [ 0.2173,  1.0973,  0.3096,  ...,  0.4675, -1.0527, -0.3312],
         [-1.6302,  1.8849, -0.8522,  ..., -1.9886, -0.3139, -2.6081],
         [ 2.8070,  1.1513, -0.6044,  ..., -1.0325,  1.2977, -0.8982]]])
tensor([[[ 0.7195,  1.1422, -0.3647,  ...,  1.9219,  0.6199,  0.9132],
         [-0.7151,  0.4172,  1.5856,  ...,  0

In [42]:
#2.多头注意力大类
class MultiHeadAttention(nn.Module):  # 继承 PyTorch 的 nn.Module
    def __init__(self, d_model, nhead, dropout=0.1):
        """
        d_model: 输入嵌入维度
        nhead: 头数（多头注意力）
        dropout: dropout 比例
        """
        super().__init__()

        # 断言确保 d_model 可以被 nhead 整除（每个头分配相同的维度）
        assert d_model % nhead == 0, "d_model must be divisible by nhead"

        self.nhead = nhead  # 记录多头数量
        self.d_k = d_model // nhead  # 每个头的维度大小

        # 定义可学习的线性变换矩阵
        self.W_q = nn.Linear(d_model, d_model)  # 查询 (Query) 权重
        self.W_k = nn.Linear(d_model, d_model)  # 键 (Key) 权重
        self.W_v = nn.Linear(d_model, d_model)  # 值 (Value) 权重
        self.W_o = nn.Linear(d_model, d_model)  # 输出 (Output) 权重

        # 定义 dropout 层
        self.dropout = nn.Dropout(dropout)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        """
        计算缩放点积注意力
        Q: 查询矩阵 (batch_size, nhead, seq_len, d_k)
        K: 键矩阵 (batch_size, nhead, seq_len, d_k)
        V: 值矩阵 (batch_size, nhead, seq_len, d_k)
        mask: 位置掩码 (batch_size, 1, 1, seq_len) 或 None
        """
        # 计算 QK^T / sqrt(d_k) (缩放点积)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        # 如果 mask 不为空，则将填充位置的得分设为一个极小值（-1e9），使其 softmax 之后趋近于 0
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        # 计算 softmax，得到注意力权重
        attn = F.softmax(scores, dim=-1)

        # 应用 dropout
        attn = self.dropout(attn)

        # 计算最终的注意力加权输出
        return torch.matmul(attn, V)

    def forward(self, Q, K, V, mask=None):
        """
        计算多头注意力
        Q, K, V: (batch_size, seq_len, d_model)
        mask: (batch_size, 1, 1, seq_len) 或 None
        """
        batch_size = Q.size(0)  # 获取 batch 大小

        # 线性变换 Q、K、V，并将其拆分为多头 (batch_size, seq_len, d_model) -> (batch_size, seq_len, nhead, d_k)
        Q = self.W_q(Q).view(batch_size, -1, self.nhead, self.d_k).transpose(1, 2)
        K = self.W_k(K).view(batch_size, -1, self.nhead, self.d_k).transpose(1, 2)
        V = self.W_v(V).view(batch_size, -1, self.nhead, self.d_k).transpose(1, 2)

        # 计算注意力
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)

        # 调整维度，使其恢复为 (batch_size, seq_len, d_model)
        attn_output = attn_output.transpose(1, 2).contiguous()
        attn_output = attn_output.view(batch_size, -1, self.nhead * self.d_k)

        # 通过最终的线性变换 W_o
        return self.W_o(attn_output)


In [43]:
#3.前馈神经网络
class PositionWiseFFN(nn.Module):  # 继承 PyTorch 的 nn.Module
    def __init__(self, d_model, d_ff, dropout=0.1):
        """
        d_model: 输入和输出的特征维度（Transformer 的嵌入维度）
        d_ff: 前馈网络中间层的维度（通常比 d_model 大很多，例如 2048）
        dropout: Dropout 比例，防止过拟合
        """
        super().__init__()

        # 定义两层全连接网络
        self.linear1 = nn.Linear(d_model, d_ff)  # 第一层，扩展特征维度
        self.linear2 = nn.Linear(d_ff, d_model)  # 第二层，将维度变回 d_model

        # Dropout 层
        self.dropout = nn.Dropout(dropout)

        # 激活函数，使用 ReLU
        self.activation = nn.ReLU()

    def forward(self, x):
        """
        前向传播
        x: 输入张量，形状为 (batch_size, seq_len, d_model)
        """
        # 计算第一层全连接 + 激活函数 + Dropout，然后通过第二层全连接
        return self.linear2(self.dropout(self.activation(self.linear1(x))))


In [44]:
# 4. Transformer 编码器层（Encoder Layer）

class EncoderLayer(nn.Module):  # 继承 PyTorch 的 nn.Module
    def __init__(self, d_model, nhead, d_ff, dropout=0.1):
        """
        d_model: 输入的嵌入维度
        nhead: 多头注意力的头数
        d_ff: 前馈神经网络的隐藏层维度
        dropout: Dropout 比例，防止过拟合
        """
        super().__init__()

        # 多头自注意力层
        self.self_attn = MultiHeadAttention(d_model, nhead, dropout)

        # 位置前馈神经网络 (FFN)
        self.ffn = PositionWiseFFN(d_model, d_ff, dropout)

        # Layer Normalization (层归一化)
        self.norm1 = nn.LayerNorm(d_model)  # 第一层归一化（针对自注意力层的输出）
        self.norm2 = nn.LayerNorm(d_model)  # 第二层归一化（针对前馈网络的输出）

        # Dropout 层，防止过拟合
        self.dropout1 = nn.Dropout(dropout)  # 用于自注意力层的输出
        self.dropout2 = nn.Dropout(dropout)  # 用于前馈网络的输出

    def forward(self, src, src_mask=None):
        """
        前向传播
        src: 输入序列张量，形状为 (batch_size, seq_len, d_model)
        src_mask: 掩码张量，形状为 (batch_size, 1, 1, seq_len)，用于防止注意力关注填充部分
        """
        # 计算多头自注意力
        src2 = self.self_attn(src, src, src, src_mask)

        # 残差连接 + Dropout + LayerNorm
        src = src + self.dropout1(src2)  # 残差连接
        src = self.norm1(src)  # 归一化

        # 计算前馈神经网络
        src2 = self.ffn(src)

        # 残差连接 + Dropout + LayerNorm
        src = src + self.dropout2(src2)  # 残差连接
        src = self.norm2(src)  # 归一化

        return src  # 返回当前编码器层的输出


In [45]:
# 5. Transformer 解码器层（Decoder Layer）
# 导入必要的库
import torch
import torch.nn as nn

class DecoderLayer(nn.Module):  # 继承 PyTorch 的 nn.Module
    def __init__(self, d_model, nhead, d_ff, dropout=0.1):
        """
        d_model: 输入的嵌入维度
        nhead: 多头注意力的头数
        d_ff: 前馈神经网络的隐藏层维度
        dropout: Dropout 比例，防止过拟合
        """
        super().__init__()

        # 多头自注意力层（解码器中的自注意力）
        self.self_attn = MultiHeadAttention(d_model, nhead, dropout)

        # 多头交叉注意力层（解码器与编码器之间的交叉注意力）
        self.cross_attn = MultiHeadAttention(d_model, nhead, dropout)

        # 位置前馈神经网络 (FFN)
        self.ffn = PositionWiseFFN(d_model, d_ff, dropout)

        # Layer Normalization (层归一化)
        self.norm1 = nn.LayerNorm(d_model)  # 第一层归一化（针对自注意力层的输出）
        self.norm2 = nn.LayerNorm(d_model)  # 第二层归一化（针对交叉注意力层的输出）
        self.norm3 = nn.LayerNorm(d_model)  # 第三层归一化（针对前馈网络的输出）

        # Dropout 层，防止过拟合
        self.dropout1 = nn.Dropout(dropout)  # 用于自注意力层的输出
        self.dropout2 = nn.Dropout(dropout)  # 用于交叉注意力层的输出
        self.dropout3 = nn.Dropout(dropout)  # 用于前馈网络的输出

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None):
        """
        前向传播
        tgt: 目标序列张量，形状为 (batch_size, tgt_seq_len, d_model)
        memory: 编码器输出的记忆张量，形状为 (batch_size, src_seq_len, d_model)
        tgt_mask: 目标序列的掩码张量，形状为 (batch_size, 1, 1, tgt_seq_len) 或 None
        memory_mask: 编码器输出的掩码张量，形状为 (batch_size, 1, 1, src_seq_len) 或 None
        """
        # 自注意力（目标序列内部的注意力）
        tgt2 = self.self_attn(tgt, tgt, tgt, tgt_mask)

        # 残差连接 + Dropout + LayerNorm
        tgt = tgt + self.dropout1(tgt2)  # 残差连接
        tgt = self.norm1(tgt)  # 归一化

        # 交叉注意力（目标序列和编码器输出之间的注意力）
        tgt2 = self.cross_attn(tgt, memory, memory, memory_mask)

        # 残差连接 + Dropout + LayerNorm
        tgt = tgt + self.dropout2(tgt2)  # 残差连接
        tgt = self.norm2(tgt)  # 归一化

        # 位置前馈网络 (FFN)
        tgt2 = self.ffn(tgt)

        # 残差连接 + Dropout + LayerNorm
        tgt = tgt + self.dropout3(tgt2)  # 残差连接
        tgt = self.norm3(tgt)  # 归一化

        return tgt  # 返回当前解码器层的输出


In [46]:
# 6. Transformer 编码器（Encoder）

class Encoder(nn.Module):  # 继承 PyTorch 的 nn.Module
    def __init__(self, vocab_size, d_model=512, nhead=8,
                 num_layers=6, d_ff=2048, dropout=0.1, max_len=5000):
        """
        vocab_size: 词汇表的大小
        d_model: 嵌入维度（输入特征的维度）
        nhead: 多头注意力的头数
        num_layers: 编码器层数
        d_ff: 前馈网络的隐藏层维度
        dropout: Dropout 比例，防止过拟合
        max_len: 最大序列长度，用于位置编码
        """
        super().__init__()

        # 嵌入层，将词索引转换为词向量
        self.embedding = nn.Embedding(vocab_size, d_model)

        # 位置编码层，增加位置位置信息
        self.pos_encoding = PositionalEncoding(d_model, max_len)

        # 编码器层，构造多个 EncoderLayer 层
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, nhead, d_ff, dropout)  # 每个 EncoderLayer 都是之前定义的
            for _ in range(num_layers)  # 构造 num_layers 个编码器层
        ])

        # Dropout 层，防止过拟合
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask=None):
        """
        前向传播
        src: 输入序列张量，形状为 (batch_size, seq_len)
        src_mask: 输入序列的掩码张量，形状为 (batch_size, 1, 1, seq_len) 或 None
        """
        # 将输入的单词索引通过嵌入层转换为词向量，并进行缩放
        src = self.embedding(src) * math.sqrt(self.embedding.embedding_dim)  # 缩放嵌入向量

        # 添加位置编码信息
        src = self.pos_encoding(src)  # 将词嵌入与位置编码相加

        # 应用 Dropout，防止过拟合
        src = self.dropout(src)

        # 通过多个 EncoderLayer 进行处理
        for layer in self.layers:
            src = layer(src, src_mask)  # 通过每个 EncoderLayer

        return src  # 返回经过多个编码器层后的输出


In [47]:
# 7. Transformer 解码器（Decoder）
class Decoder(nn.Module):  # 继承 PyTorch 的 nn.Module
    def __init__(self, vocab_size, d_model=512, nhead=8,
                 num_layers=6, d_ff=2048, dropout=0.1, max_len=5000):
        """
        vocab_size: 词汇表的大小
        d_model: 嵌入维度（输入特征的维度）
        nhead: 多头注意力的头数
        num_layers: 解码器层数
        d_ff: 前馈网络的隐藏层维度
        dropout: Dropout 比例，防止过拟合
        max_len: 最大序列长度，用于位置编码
        """
        super().__init__()

        # 目标序列嵌入层，将词索引转换为词向量
        self.embedding = nn.Embedding(vocab_size, d_model)

        # 位置编码层，增加位置位置信息
        self.pos_encoding = PositionalEncoding(d_model, max_len)

        # 解码器层，构造多个 DecoderLayer 层
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, nhead, d_ff, dropout)  # 每个 DecoderLayer 都是之前定义的
            for _ in range(num_layers)  # 构造 num_layers 个解码器层
        ])

        # Dropout 层，防止过拟合
        self.dropout = nn.Dropout(dropout)

        # 输出投影层，将解码器的输出映射回词汇表大小
        self.projection = nn.Linear(d_model, vocab_size)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None):
        """
        前向传播
        tgt: 目标序列张量，形状为 (batch_size, tgt_seq_len)
        memory: 编码器输出的记忆张量，形状为 (batch_size, src_seq_len, d_model)
        tgt_mask: 目标序列的掩码张量，形状为 (batch_size, 1, 1, tgt_seq_len) 或 None
        memory_mask: 编码器输出的掩码张量，形状为 (batch_size, 1, 1, src_seq_len) 或 None
        """
        # 将输入的目标序列单词索引转换为词向量，并进行缩放
        tgt = self.embedding(tgt) * math.sqrt(self.embedding.embedding_dim)  # 缩放嵌入向量

        # 添加位置编码信息
        tgt = self.pos_encoding(tgt)  # 将词嵌入与位置编码相加

        # 应用 Dropout，防止过拟合
        tgt = self.dropout(tgt)

        # 通过多个 DecoderLayer 进行处理
        for layer in self.layers:
            tgt = layer(tgt, memory, tgt_mask, memory_mask)  # 通过每个 DecoderLayer

        # 通过投影层将解码器的输出映射回词汇表大小
        return self.projection(tgt)  # 形状为 (batch_size, tgt_seq_len, vocab_size)



In [48]:
# 8. Transformer 模型
class Transformer(nn.Module):  # 继承 PyTorch 的 nn.Module
    def __init__(self, src_vocab_size, tgt_vocab_size,
                 d_model=512, nhead=8, num_layers=6,
                 d_ff=2048, dropout=0.1, max_len=5000):
        """
        src_vocab_size: 源语言的词汇表大小
        tgt_vocab_size: 目标语言的词汇表大小
        d_model: 嵌入维度（输入特征的维度）
        nhead: 多头注意力的头数
        num_layers: 编码器和解码器的层数
        d_ff: 前馈神经网络的隐藏层维度
        dropout: Dropout 比例，防止过拟合
        max_len: 最大序列长度，用于位置编码
        """
        super().__init__()

        # 定义 Transformer 的编码器
        self.encoder = Encoder(src_vocab_size, d_model, nhead,
                              num_layers, d_ff, dropout, max_len)

        # 定义 Transformer 的解码器
        self.decoder = Decoder(tgt_vocab_size, d_model, nhead,
                              num_layers, d_ff, dropout, max_len)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        """
        前向传播
        src: 源序列张量，形状为 (batch_size, src_seq_len)
        tgt: 目标序列张量，形状为 (batch_size, tgt_seq_len)
        src_mask: 源序列的掩码张量，形状为 (batch_size, 1, 1, src_seq_len) 或 None
        tgt_mask: 目标序列的掩码张量，形状为 (batch_size, 1, 1, tgt_seq_len) 或 None
        """
        # 通过编码器处理源序列
        memory = self.encoder(src, src_mask)  # 形状: (batch_size, src_seq_len, d_model)

        # 通过解码器处理目标序列，结合编码器的输出
        output = self.decoder(tgt, memory, tgt_mask)  # 形状: (batch_size, tgt_seq_len, vocab_size)

        return output  # 返回解码器的最终输出


In [49]:
# 9. mask生成函数

def generate_mask(src, tgt, pad_idx=0):
    """
    生成 Transformer 所需的掩码（mask）。

    参数：
    src: 源序列张量，形状为 (batch_size, src_seq_len)
    tgt: 目标序列张量，形状为 (batch_size, tgt_seq_len)
    pad_idx: 填充索引（默认为 0），用于标记填充部分

    返回：
    src_mask: 源序列的掩码，形状为 (batch_size, 1, 1, src_seq_len)
    tgt_mask: 目标序列的掩码，形状为 (batch_size, 1, tgt_seq_len, tgt_seq_len)
    """

    # 生成源序列的 Padding Mask：用于防止模型关注填充 (PAD) 位置
    # (src != pad_idx) 生成一个布尔张量，标记非 PAD 位置为 True，PAD 位置为 False
    # 通过 unsqueeze(1) 和 unsqueeze(2) 扩展维度，使其形状变为 (batch_size, 1, 1, src_seq_len)
    src_mask = (src != pad_idx).unsqueeze(1).unsqueeze(2)  # 形状: (B, 1, 1, S)

    # 生成目标序列的 Padding Mask
    # 形状: (batch_size, 1, tgt_seq_len, 1)
    tgt_pad_mask = (tgt != pad_idx).unsqueeze(1).unsqueeze(3)

    # 计算目标序列的 Look-Ahead Mask (防止解码器看到未来的信息)
    seq_len = tgt.size(1)  # 目标序列长度
    look_ahead_mask = torch.ones(seq_len, seq_len).tril().bool()  # 生成一个下三角矩阵

    # 由于 Look-Ahead Mask 应该应用到整个 batch 上，因此需要扩展维度
    # (1, 1, tgt_seq_len, tgt_seq_len) -> 适用于 batch 维度
    look_ahead_mask = look_ahead_mask.unsqueeze(0).unsqueeze(0)

    # 目标序列的总掩码 = Padding Mask & Look-Ahead Mask
    # 只有当 tgt_pad_mask 和 look_ahead_mask 都为 True 时，模型才能关注该位置
    tgt_mask = tgt_pad_mask & look_ahead_mask.to(tgt.device)  # 形状: (B, 1, T, T)

    return src_mask, tgt_mask


In [58]:

# 定义源语言和目标语言的词汇表大小
src_vocab = 10000  # 源语言词汇表大小
tgt_vocab = 8000   # 目标语言词汇表大小

# 创建 Transformer 模型，传入词汇表大小和其他超参数
model = Transformer(src_vocab, tgt_vocab)

# 模拟输入，假设每个序列的长度是 4，batch_size 是 2
src = torch.LongTensor([[1, 2, 3, 4], [5, 6, 0, 0]])  # 0是padding，形状为 (batch_size, seq_len)
tgt = torch.LongTensor([[10, 20, 30, 40], [10, 60, 70, 0]])  # 同样，0是padding，形状为 (batch_size, seq_len)

# 生成源序列和目标序列的掩码（mask），用于在计算注意力时防止模型关注填充位置
src_mask, tgt_mask = generate_mask(src, tgt)
print(tgt_mask)


output = model(src, tgt, src_mask, tgt_mask)

# 打印输出的形状
print("输出形状:", output.shape)  # 预期输出形状是 (batch_size, tgt_seq_len-1, vocab_size)，即 (2, 3, 8000)


tensor([[[[ True, False, False, False],
          [ True,  True, False, False],
          [ True,  True,  True, False],
          [ True,  True,  True,  True]]],


        [[[ True, False, False, False],
          [ True,  True, False, False],
          [ True,  True,  True, False],
          [False, False, False, False]]]])
输出形状: tensor([ 0.7169, -0.1168,  1.2205,  ..., -0.0685,  0.1231,  0.1344],
       grad_fn=<SelectBackward0>)
